# Load Bridges2 Modules and Set Environment Variables

In [ ]:
%set_env SHELL=/bin/bash

In [ ]:
import sys
sys.path.append("/usr/share/lmod/lmod/init")
sys.path.append("/jet/home/wnw36/.conda/envs/icomse-dft/lib/python3.1/site-packages/")
from env_modules_python import module
module('load', 'intelmpi')
module('load', 'QuantumEspresso')

In [ ]:
import os
os.environ['ASE_ESPRESSO_COMMAND'] = "mpirun -np 4 pw.x -in PREFIX.pwi > PREFIX.pwo"
os.environ['OMP_NUM_THREADS'] = "2"

# SCF Calculation for Metallic Surfaces

In [ ]:
from ase.visualize import view
from ase.build import fcc111

atoms = fcc111('Al', size=(2,2,4), vacuum=7.5) # Vacuum added on both sides! 15 Ang. Vacuum total
view(atoms, viewer="x3d")

In [ ]:
from ase.calculators.espresso import Espresso

pseudopotentials = {
    "Al":"Al.pbe-n-kjpaw_psl.1.0.0.UPF"
}

input_data = {
    'system': {
        'ecutwfc': 20,
        'ecutrho': 160,
        'occupations': 'smearing',
        'smearing':'mp',
        'degauss':0.02
    },
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos'
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta':0.2,
        'mixing_mode': 'local-TF'
    }
} 


calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(4,4,1),
    koffset=(1, 1, 0))

atoms.calc = calc


In [ ]:
energy = atoms.get_potential_energy()
print(f"E = {energy:.3f}")

# Geometry Optimization

In [ ]:
input_data['control'].update({'restart_mode':'restart'})
calc.set(input_data = input_data)

In [ ]:
from ase.optimize import BFGS
from ase.constraints import FixAtoms

c = FixAtoms(indices=[atom.index for atom in atoms if atom.position[2] <= 0.5*atoms.get_cell()[2][2]])
atoms.set_constraint(c)

opt = BFGS(atoms, trajectory="bfgs.traj")
opt.run(fmax=0.05)